In [46]:
file = open('imdb_train.txt','r',encoding='utf-8')
label = []
feature = []

l = 0
for line in file:
    label.append(line[0])
    feature.append(line[2:])

    
for i in range(50):
    print(label[i])
for i in range(50):
    print(feature[i])


1
0
0
1
0
0
1
0
1
0
1
0
0
0
0
0
1
1
0
1
0
0
1
0
1
1
0
0
1
0
0
1
1
1
0
0
0
1
0
0
1
0
0
0
1
1
1
0
1
0
this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them a

In [57]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



UnicodeDecodeError: 'cp932' codec can't decode byte 0x96 in position 4384: illegal multibyte sequence